In [1]:
import os
import h5py
import cv2
import numpy as np
import pandas as pd
import sklearn as skl
from matplotlib import pyplot as plt

from pdf2image import convert_from_path, convert_from_bytes
import pytesseract

from OCRPage import OCRPage

In [2]:
# set home directory
home_path = os.getcwd()[:os.getcwd().find('Code/python/obf_autax/ocr')-1]
# set input_path
input_path = os.path.join(home_path, 'Data', 'obf', 'Haselbach_Mannersdorf_30042020.pdf')
#Operat Hinterriß

In [3]:
def convert_to_image(input_path, page_nr):

    pages = convert_from_path(input_path, 500, first_page=page_nr, last_page=page_nr)

    return pages[0]

In [4]:
img = convert_to_image(input_path, 3).convert('RGB')
img = np.array(img)

img = convert_to_image(input_path, 113).convert('RGB')
img = np.array(img)
ocr = OCRPage(img)
ocr.convert()

## Text

In [5]:
# get text
txt = ocr.get_text('text')

In [6]:
# split lines
txt = txt.splitlines()

In [7]:
txt

['ST im Süden breiter Rücken, am Rücken steinig mit STOE 71 und HD-Typ',
 'BE einige ES KB BU BI AH Kl, Elam Rücken, einige eingewachsene BU-El-Überhälter, ungleichaltrig +/- 10 Jahre, Ausschlagswald']

In [8]:
txt[1].replace('El', 'EI')

'BE einige ES KB BU BI AH Kl, EIam Rücken, einige eingewachsene BU-EI-Überhälter, ungleichaltrig +/- 10 Jahre, Ausschlagswald'

In [58]:
txt

['ST im Süden breiter Rücken, am Rücken steinig mit STOE 71 und HD-Typ',
 'BE einige ES KB BU BI AH Kl, Elam Rücken, einige eingewachsene BU-El-Überhälter, ungleichaltrig +/- 10 Jahre, Ausschlagswald']

In [59]:
len(txt)

2

In [61]:
for i in range(3-2):
    txt.append("")

In [62]:
l0, l1, l2 = txt

## Maßnahme

In [31]:
ma = ocr.get_text('mass')
spaces = ocr.get_spaces()

In [46]:
def from_string(ma_as_str, spaces):
    # split lines
    ma = ma_as_str.splitlines()
    l = []
    ch = ""

    for i in range(len(ma[0])):

        if i == 0:
            ch = ma[0][i]
        elif i == len(ma[0])-1:
            l.append(ch + ma[0][i])
        elif spaces[0][i-1] == 0:
            ch = ch + ma[0][i]
        elif spaces[0][i-1] == 1:
            l.append(ch)
            ch = ma[0][i]
    l.append(ma[1])
    return l

In [39]:
from_string(ma)

['DF', '4,7', '300', '0', '2', '3', '4', '35', 'El Kronen freistellen']

In [45]:
spaces

[array([0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0])]

In [47]:
ma = 'DF4,7300023435\nEl Kronen freistellen'
spaces = [np.array([0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0])]
from_string(ma, spaces)

['DF', '4,7', '300', '0', '2', '3', '4', '35', 'El Kronen freistellen']

In [49]:
ma = 'DF47300023435\nEl Kronen freistellen'
spaces = [np.array([0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0])]
from_string(ma, spaces)

['DF', '473', '000', '2', '3', '4', '35', 'El Kronen freistellen']

In [50]:
# added 1 to spaces and ma
ma = 'DF4,73000213435\nEl Kronen freistellen'
spaces = [np.array([0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0])]
from_string(ma, spaces)

['DF', '4,7', '300', '0', '2', '1', '3', '4', '35', 'El Kronen freistellen']

In [51]:
# added 1 to spaces and ma
ma = 'DF213435\nEl Kronen freistellen'
spaces = [np.array([0, 1, 1, 1, 1, 1, 1, 1, 0])]
from_string(ma, spaces)

['DF', '2', '1', '3', '4', '35', 'El Kronen freistellen']

In [9]:
# split lines
ma = ma.splitlines()

In [10]:
ma

['DF4,7300023435', 'El Kronen freistellen']

In [44]:
l = []
ch = ""

for i in range(len(ma[0])):
    
    if i == 0:
        ch = ma[0][i]
    elif i == len(ma[0])-1:
        l.append(ch + ma[0][i])
    elif spaces[0][i-1] == 0:
        ch = ch + ma[0][i]
    elif spaces[0][i-1] == 1:
        l.append(ch)
        ch = ma[0][i]
l.append(ma[1])

In [45]:
l

['DF', '4,7', '300', '0', '2', '3', '4', '35', 'El Kronen freistellen']

## BZ

In [10]:
bz = "3BU 4FI 3AH"

In [11]:
bz = bz.split()

In [12]:
bz

['3BU', '4FI', '3AH']

In [13]:
bz[0][-2:]

'BU'

In [14]:
int(bz[0][:-2])

3

In [18]:
bzs = []
for b in bz:
    bzs.append([int(b[:-2]),b[-2:]])

In [20]:
# Return double of n 
def addition(n): 
    return n + n 
  
# We double all numbers using map() 
numbers = (1, 2, 3, 4) 
result = map(addition, numbers) 
print(list(result))

[2, 4, 6, 8]


In [21]:
bz

['3BU', '4FI', '3AH']

In [27]:
def spliter(n):
    return int(n[:-2]),n[-2:]
result = map(spliter, bz)

In [28]:
result

In [19]:
bzs

[[3, 'BU'], [4, 'FI'], [3, 'AH']]

In [1]:
from OCRData import BestZiel
from OCRData import BAZiel

In [2]:
ba1 = BAZiel(13, 'BU')
ba2 = BAZiel(4, 'K')
ba3 = BAZiel(3, 'LA')

In [3]:
bz = BestZiel([ba1, ba2, ba3])

In [8]:
bz = BestZiel.from_string('3BI 4KI 3FI')

3BI
3
BI
4KI
4
KI
3FI
3
FI


In [9]:
bz

BestZiel(bz=[BAZiel(part=3, ba='BI', error=False), BAZiel(part=4, ba='KI', error=False), BAZiel(part=3, ba='FI', error=False)], error=False)

In [10]:
bz.check_attr()

In [11]:
bz

BestZiel(bz=[BAZiel(part=3, ba='BI', error=False), BAZiel(part=4, ba='KI', error=False), BAZiel(part=3, ba='FI', error=False)], error=False)

In [6]:
from OCRData import Text

In [9]:
# get text
txt = ocr.get_text('text')

In [10]:
Text.from_string(txt)

Text(line0='ST im Süden breiter Rücken, am Rücken steinig mit STOE 71 und HD-Typ', line1='BE einige ES KB BU BI AH Kl, Elam Rücken, einige eingewachsene BU-El-Überhälter, ungleichaltrig +/- 10 Jahre, Ausschlagswald', line2='')

In [24]:
from OCRData import Nutz

In [46]:
nutz = Nutz(ma=l[0], v_lh=int(l[2]), v_nh=int(l[3]), dring=int(l[4]), behoerde=0, zeitp=int(l[5]), \
            schlaeg=int(l[6]), rueck=int(l[7]), text=l[8])

In [47]:
l

['DF', '4,7', '300', '0', '2', '3', '4', '35', 'El Kronen freistellen']

In [48]:
nutz

Nutz(ma='DF', area=0.0, v_lh=300, v_nh=0, dring=2, behoerde=0, zeitp=3, schlaeg=4, rueck=35, text='El Kronen freistellen', error=False)

In [49]:
nutz.check_attr()

In [50]:
nutz

Nutz(ma='DF', area=0.0, v_lh=300, v_nh=0, dring=2, behoerde=0, zeitp=3, schlaeg=4, rueck=35, text='El Kronen freistellen', error=True)